<a href="https://colab.research.google.com/github/jameschartouni/arabic_translation/blob/xlm/XLM_Bash_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#installation step
!pip install transformers
!pip install sentencepiece
!pip install bpemb
#creating the folders 
!mkdir data/
!mkdir data/AD_NMT-master
!mkdir data/train/
!mkdir data/test/
!mkdir data/val/
!mkdir data/vocab
!mkdir data/model

#fetching the pkl files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW' -O data/AD_NMT-master/english-Arabic-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V8_tp8ZlWUYaX7QQL46t0uSRNrVehSf1' -O data/AD_NMT-master/english-Arabic-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7X0qtuDIyjTHY0wh-ZNoVwsiF4lId2e' -O data/AD_NMT-master/english-Arabic-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UzL4cOWTMCee83KBUh2QO_H62AFVpDQV' -O data/AD_NMT-master/LAV-MSA-2-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UpfCbkxhztof7dvNjeAs1bHjD4SER6h3' -O data/AD_NMT-master/LAV-MSA-2-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlAZGtYsSfXzK7hrC_PbxQFqTSXD0DMw' -O data/AD_NMT-master/LAV-MSA-2-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UjDX7cCG2S23SPfSHxSPdVayMTxB5Y16' -O data/AD_NMT-master/Magribi_MSA-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UaVWIqRXo0rxuxDF4KArA4bEK1TaLX3l' -O data/AD_NMT-master/Magribi_MSA-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYvlhdYAdfa4riP_4hn3-IEVd1ZUXVTQ' -O data/AD_NMT-master/Magribi_MSA-train.pkl

     |████████████████████████████████| 645kB 2.6MB/s 
     |████████████████████████████████| 1.1MB 12.8MB/s 
     |████████████████████████████████| 3.8MB 18.1MB/s 
     |████████████████████████████████| 890kB 39.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=09cbfc854f60f667895f07da0c16f1bedc955ee9c2cb43e7c60155a3d625c540
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
--2020-05-13 13:05:04--  https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW
Resolving docs.google.com (docs.google.com)... 74.125.203.100, 74.125.203.138, 74.125.203.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.203.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tkhaonisjqqh8ufm92lmca6nalinh

In [0]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
from torch.optim import SGD
from torchtext import data
import pandas as pd
import numpy as np
import pickle
import os, io, glob
import functools
from sklearn.model_selection import train_test_split

from bpemb import BPEmb

import transformers
from transformers import XLMTokenizer, XLMModel

In [0]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_English_MSA_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_English_MSA_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [0]:
print(data_English_MSA_both[0:5])
print(data_LAV_MSA_both[0:5])
print(len(data_English_MSA_trainval))
print(len(data_English_MSA_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


##Prepare Datasets

In [0]:
#bpemb_en = BPEmb(lang="en", dim=50)
#bpemb_ar = BPEmb(lang="ar", dim=50)
multibpemb = BPEmb(lang="multi", vs=1000000, dim=300)

downloading https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs1000000.model


100%|██████████| 20636145/20636145 [00:03<00:00, 5263203.80B/s]


downloading https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs1000000.d300.w2v.bin.tar.gz


100%|██████████| 1123372891/1123372891 [01:56<00:00, 9667578.39B/s] 
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
xlmr = torch.hub.load('pytorch/fairseq', 'xlmr.large')

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_master
100%|██████████| 1028340964/1028340964 [01:10<00:00, 14615738.56B/s]


In [0]:
from fairseq.models.roberta import XLMRModel
xlmr_fair = XLMRModel.from_pretrained('xlmr.large', checkpoint_file='model.pt')


In [0]:
%%writefile xlm_train.sh
python train.py

## main parameters
--exp_name ar-en                                       # experiment name
--dump_path ./dumped/                                         # where to store the experiment
--reload_model 'xlmr_large.pth,xlmr_large.pth'          # model to reload for encoder,decoder

## data location / training objective
--data_path ./data/train/                           # data location
--lgs 'en-ar'                                                 # considered languages
--mt_steps "en-ar,ar-en"                                            # denoising auto-encoder training steps
--bt_steps 'en-ar-en,ar-en-ar'                                # back-translation steps
--word_shuffle 3                                              # noise for auto-encoding loss
--word_dropout 0.1                                            # noise for auto-encoding loss
--word_blank 0.1                                              # noise for auto-encoding loss
--lambda_ae '0:1,100000:0.1,300000:0'                         # scheduling on the auto-encoding coefficient

## transformer parameters
--encoder_only false                                          # use a decoder for MT
--emb_dim 1024                                                # embeddings / model dimension
--n_layers 6                                                  # number of layers
--n_heads 8                                                   # number of heads
--dropout 0.1                                                 # dropout
--attention_dropout 0.1                                       # attention dropout
--gelu_activation true                                        # GELU instead of ReLU

## optimization
--tokens_per_batch 2000                                       # use batches with a fixed number of words
--batch_size 32                                               # batch size (for back-translation)
--bptt 256                                                    # sequence length
--optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001  # optimizer
--epoch_size 200000                                           # number of sentences per epoch
--eval_bleu true                                              # also evaluate the BLEU score
--stopping_criterion 'valid_en-fr_mt_bleu,10'                 # validation metric (when to save the best model)
--validation_metrics 'valid_en-fr_mt_bleu'                    # end experiment if stopping criterion does not improve

Writing xlm_train.sh


In [0]:
!bash xlm_train.sh

FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
Traceback (most recent call last):
  File "train.py", line 17, in <module>
    from src.trainer import SingleTrainer, EncDecTrainer
  File "/content/XLM/src/trainer.py", line 18, in <module>
    import apex
ModuleNotFoundError: No module named 'apex'
xlm_train.sh: line 4: --exp_name: command not found
xlm_train.sh: line 5: --dump_path: command not found
xlm_train.sh: line 6: --reload_model: command not found
xlm_train.sh: line 9: --data_path: command not found
xlm_train.sh: line 10: --lgs: command not found
xlm_train.sh: line 11: --mt_steps: command not found
xlm_train.sh: line 12: --bt_steps: command not found
xlm_train.sh: line 13: --word_shuffle: command not found
xlm_train.sh: line 14: --word_dropout: command not found
xlm_train.sh: line 15: --word_blank: command not found
xlm_train.sh: line 16: --lambda_ae: command not found
xlm_train.sh: line 19: --encoder_only: comman

In [0]:
#splits the train dataset into train and validation sets, define test set as datafile
msa_en_train, msa_en_val = train_test_split(data_English_MSA_trainval, test_size=.2, random_state=22)
msa_en_test = data_English_MSA_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2, random_state=22)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2, random_state=22)
mag_msa_test = data_Magribi_MSA_test

In [0]:
file_path = 'data/'

def list_to_pth(ds, src='msa', trg='en', datatype=''):
    src_formatted = datatype + '.' + src + '-' + trg + '.' + 'pth'
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as pth:
        for i, arr in enumerate(ds):
            pth.write(multibpemb.encode_ids_with_bos_eos(arr[1]) + ',' + multibpemb.encode_ids_with_bos_eos(arr[0]) + '\n')

In [0]:
#splits each language pair file into datasets of single language, to be merged again by the pytorch dataset class later
list_to_pth(msa_en_train, 'msa', 'en', 'train')
list_to_pth(msa_en_val, 'msa', 'en', 'val')
list_to_pth(msa_en_test, 'msa', 'en', 'test')

list_to_pth(lav_msa_train, 'lav', 'msa', 'train')
list_to_pth(lav_msa_val, 'lav', 'msa', 'val')
list_to_pth(lav_msa_test, 'lav', 'msa', 'test')

list_to_pth(mag_msa_train, 'mag', 'msa', 'train')
list_to_pth(mag_msa_val, 'mag', 'msa', 'val')
list_to_pth(mag_msa_test, 'mag', 'msa', 'test')

TypeError: ignored

##BPemb Tokenizer

In [0]:
file_path = 'data/'
def csv_to_pth(ds, src='msa', trg='en', datatype=''):
    src_formatted = datatype + '.' + src + '-' + trg + '.' + 'pth'
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as pth:
        for i, arr in enumerate(ds):
            pth.write(multibpemb.encode_ids_with_bos_eos(arr[1]) + ',' + multibpemb.encode_ids_with_bos_eos(arr[0]))

In [0]:
csv_to_pth('data/train/train_msa_en.csv', 'msa', 'en', 'train')
csv_to_pth('data/val/val_msa_en.csv', 'msa', 'en', 'val')
csv_to_pth('data/test/test_msa_en.csv', 'msa', 'en', 'test')

csv_to_pth('data/train/train_lav_msa.csv', 'lav', 'msa', 'train')
csv_to_pth('data/val/val_lav_msa.csv', 'lav', 'msa', 'val')
csv_to_pth('data/test/test_lav_msa.csv', 'lav', 'msa', 'test')

csv_to_pth('data/train/train_mag_msa.csv', 'mag', 'msa', 'train')
csv_to_pth('data/val/val_mag_msa.csv', 'mag', 'msa', 'val')
csv_to_pth('data/test/test_mag_msa.csv', 'mag', 'msa', 'test')

IndexError: ignored

In [0]:
filepath = str(multibpemb.words)
print(filepath)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
!filepath='https://nlp.h-its.org/bpemb/multi/multi.wiki.bpe.vs100000.vocab'
!DATA_PATH='data/'

In [0]:
!./preprocess.py python $filepath $DATA_PATH/train/train_msa_en.csv

Traceback (most recent call last):
  File "./preprocess.py", line 31, in <module>
    assert os.path.isfile(voc_path)
AssertionError


In [0]:
for i in data_English_MSA_both:
  en_encode = multibpemb.encode_ids_with_eos(i[0])
  print(en_encode)

Streaming output truncated to the last 5000 lines.
[46, 6896, 12, 1746, 2]
[1308, 994781, 11826, 7754, 20356, 254344, 2]
[2849, 994781, 994765, 509712, 9051, 2]
[46, 994781, 994770, 288, 2260, 9363, 11908, 2]
[46, 994781, 477, 841334, 38282, 82572, 120273, 2]
[46, 994781, 477, 9857, 15997, 19097, 2]
[7588, 276, 5385, 54108, 5385, 2783, 995602, 2]
[1308, 1827, 985, 57, 12, 15465, 4700, 2]
[46, 6896, 174, 7934, 217, 192286, 2]
[200, 2849, 87063, 995602, 2]
[73378, 2783, 31071, 290744, 386, 144942, 995602, 2]
[46, 249, 60, 12, 1551, 8764, 2]
[25371, 24987, 2]
[46, 2178, 994781, 994764, 86675, 86641, 2]
[46, 994781, 994770, 13396, 171947, 300, 32180, 2]
[12317, 94, 25519, 995333, 2]
[2397, 7793, 46, 994781, 994770, 3575, 514274, 5385, 994781, 106, 15548, 1551, 432859, 48634, 994781, 994764, 5385, 995602, 2]
[2230, 108288, 174, 98439, 201990, 2036, 10559, 2]
[46, 994781, 994770, 3575, 45271, 33619, 2]
[118319, 9018, 456, 300, 406, 24987, 2]
[2783, 5385, 31071, 332, 40973, 995602, 2]
[580, 9

Pytorch Dataset

In [0]:
#build custom pytorch dataset class

#BucketIteterator code 
#https://pytorch.org/text/_modules/torchtext/data/iterator.html

In [0]:
!ls data/

AD_NMT-master  model  test  train  val	vocab


In [0]:
class CustomTranslationDataset(data.Dataset):

  '''Create a CustomTranslationDataset Class

  Arguements:
    path: Common prefix of paths to the data files for both languages 
    exts: a tuple containing the extension to path for each language 

  '''


  def __init__(self, path, exts):

    src_path, trg_path = tuple(os.path.expanduser(path + x) for x in exts)
    print(src_path, trg_path)

    examples = []
    with io.open(src_path, mode='r', encoding='utf-8') as src_file, io.open(trg_path, mode='r', encoding='utf-8') as trg_file:
      for src_line, trg_line in zip(src_file, trg_file):
                src_line, trg_line = src_line.strip(), trg_line.strip()
                if src_line != '' and trg_line != '':
                    examples.append(data.Example.fromlist(
                        [src_line, trg_line]))
                    
  

  @classmethod
  def splits(cls, exts, path=None, root='.data',
              train='train', validation='val', test='test', **kwargs):
      """Create dataset objects for splits of a TranslationDataset.
      Arguments:
          exts: A tuple containing the extension to path for each language.
          fields: A tuple containing the fields that will be used for data
              in each language.
          path (str): Common prefix of the splits' file paths, or None to use
              the result of cls.download(root).
          root: Root dataset storage directory. Default is '.data'.
          train: The prefix of the train data. Default: 'train'.
          validation: The prefix of the validation data. Default: 'val'.
          test: The prefix of the test data. Default: 'test'.
          Remaining keyword arguments: Passed to the splits method of
              Dataset.
      """
      if path is None:
          path = cls.download(root)

      train_data = None if train is None else cls(
          os.path.join(path, train), exts, **kwargs)
      val_data = None if validation is None else cls(
          os.path.join(path, validation), exts, **kwargs)
      test_data = None if test is None else cls(
          os.path.join(path, test), exts, **kwargs)
      return tuple(d for d in (train_data, val_data, test_data)
                    if d is not None)



In [0]:
eng_msa_train_dataset = CustomTranslationDataset(path='data/train/train_eng_msa.', exts=('eng', 'msa'))
lav_msa_train_dataset = CustomTranslationDataset(path='data/train/train_lav_msa.', exts=('lav', 'msa'))
mag_msa_train_dataset = CustomTranslationDataset(path='data/train/train_mag_msa.', exts=('mag', 'msa'))

data/train/train_eng_msa.eng data/train/train_eng_msa.msa
data/train/train_lav_msa.lav data/train/train_lav_msa.msa
data/train/train_mag_msa.mag data/train/train_mag_msa.msa


In [0]:
#train, validation and test are probably wrong.  
#https://github.com/pytorch/text/blob/master/torchtext/datasets/translation.py

'''
Arguments:
            exts: A tuple containing the extension to path for each language.
            fields: A tuple containing the fields that will be used for data
                in each language.
            path (str): Common prefix of the splits' file paths, or None to use
                the result of cls.download(root).
            root: Root dataset storage directory. Default is '.data'.
            train: The prefix of the train data. Default: 'train'.
            validation: The prefix of the validation data. Default: 'val'.
            test: The prefix of the test data. Default: 'test'.
            Remaining keyword arguments: Passed to the splits method of
                Dataset.

'''


train_data, valid_data, test_data = eng_msa_train_dataset.splits(path= 'data/', train='train/train_eng_msa', validation='val/val_eng_msa', test='test/test_eng_msa', exts=('.eng', '.msa'))

In [0]:
BATCH_SIZE =32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device,
    shuffle=True)